In [62]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *
from typing import Tuple, List, Optional

from bubble_api import Field as BubbelField

import openai
import instructor

In [2]:
prompts_path = "Prompts/fr/"

In [5]:
print(prompt_company_informations.text)

Je cherche a obtenir une description de l'entreprise Darty en restant aussi neutre que possible et en mettand l'accent sur les évènements récents. 
Cette description sera utilisé pour informer des gens en charge d'analyser les retours des clients de l'entreprise sur ces dernières années. 
Ne parle pas ce ce que les retours clients pourraient donner, afin de ne pas biaiser l'étude. 

tu devras me donner:
- Une courte description de l'entreprise
- La liste des évènement récents de l'entreprise, composés d'une date et d'une phrase de description.
- La liste des principaux compétiteurs de l'entreprise.
- La liste des principeux défis auquels fait face l'entreprise.



In [15]:
test = CompanyContext(description="desc", recent_events=[("1900", "rien")], competitors=["moi"], challenges=['aucun'])
print(str(test))

Description:
desc

Evènement récents:
1900 : rien

Concurrents:
moi

Défis:
aucun


In [16]:
test.model_json_schema()

{'properties': {'description': {'description': "Description de l'entreprise.",
   'title': 'Description',
   'type': 'string'},
  'recent_events': {'description': "Liste des évènement récents de l'entreprise, avec leur date une phrase de description.",
   'items': {'maxItems': 2,
    'minItems': 2,
    'prefixItems': [{'type': 'string'}, {'type': 'string'}],
    'type': 'array'},
   'title': 'Recent Events',
   'type': 'array'},
  'competitors': {'description': "Liste des principaux compétiteurs de l'entreprise.",
   'items': {'type': 'string'},
   'title': 'Competitors',
   'type': 'array'},
  'challenges': {'description': "Liste des défis auquels fait face l'entreprise.",
   'items': {'type': 'string'},
   'title': 'Challenges',
   'type': 'array'}},
 'required': ['description', 'recent_events', 'competitors', 'challenges'],
 'title': 'CompanyContext',
 'type': 'object'}

In [17]:
class Txt(BaseModel):
    rep: str

get_analysis("Dis Bonjour", Txt)

Txt(rep='Bonjour')

In [26]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class Event(BaseModel):
    date: str
    description: str

class CompanyContext(BaseModel):
    description: str = Field(description="Description de l'entreprise.")
    recent_events: List[Event] = Field(description="Liste des évènement récents de l'entreprise, avec leur date une phrase de description.")
    competitors: List[str] = Field(description="Liste des principaux compétiteurs de l'entreprise.")
    challenges: List[str] =  Field(description="Liste des défis auquels fait face l'entreprise.")

    def __str__(self):
        return '\n'.join(["Description:\n"+self.description] + ['\nEvènement récents:'] + [ev.date+' : '+ev.description for ev in self.recent_events] + ['\nConcurrents:'] + self.competitors + ['\nDéfis:'] + self.challenges)
        #return '\n'.join(["Description:\n"+self.description] + ['\nEvènements récents:'] +  self.recent_events + ['\nConcurrents:'] + self.competitors + ['\nDéfis:'] + self.challenges)

test = CompanyContext(description="desc", recent_events=[Event(date="1900", description="rien")], competitors=["moi"], challenges=['aucun'])
print(str(test))

Description:
desc

Evènement récents:
1900 : rien

Concurrents:
moi

Défis:
aucun


In [27]:

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = str(res)
print(res)

Description:
Darty est une chaîne de magasins française spécialisée dans la vente de matériel informatique, de téléviseurs, d'électroménager, de matériel audio et vidéo. Fondée en 1957, l'entreprise s'est développée pour devenir l'un des leaders du marché français dans son secteur. Darty est reconnue pour son service après-vente, symbolisé par le 'Contrat de Confiance' qui promet une assistance et des garanties étendues à ses clients.

Evènement récents:
2022-11 : Darty annonce un partenariat avec Google pour intégrer l'assistant vocal Google Assistant dans une gamme de produits électroménagers connectés.
2023-02 : Lancement d'une nouvelle campagne publicitaire mettant en avant le recyclage des appareils électroniques et électroménagers en magasin.
2023-03 : Ouverture d'un nouveau magasin phare à Paris, visant à offrir une expérience client améliorée avec des zones de démonstration interactives.

Concurrents:
Boulanger
Fnac
Conforama
Amazon

Défis:
Adaptation aux nouvelles technologies

In [42]:
res.description

"Darty est une chaîne de magasins française spécialisée dans la vente de matériel informatique, de téléviseurs, d'électroménager, de matériel audio et vidéo. Fondée en 1957, l'entreprise s'est développée pour devenir l'un des leaders du marché français dans son secteur. Darty est reconnue pour son service après-vente, symbolisé par le 'Contrat de Confiance' qui promet une assistance et des garanties étendues à ses clients."

In [39]:
context

{'company': 'Darty',
 'project': 'Darty_trustpilot',
 'context': "Description:\nDarty est une chaîne de magasins française spécialisée dans la vente de matériel informatique, de téléviseurs, d'électroménager, de matériel audio et vidéo. Fondée en 1957, l'entreprise s'est développée pour devenir l'un des leaders du marché français dans son secteur. Darty est reconnue pour son service après-vente, symbolisé par le 'Contrat de Confiance' qui promet une assistance et des garanties étendues à ses clients.\n\nEvènement récents:\n2022-11 : Darty annonce un partenariat avec Google pour intégrer l'assistant vocal Google Assistant dans une gamme de produits électroménagers connectés.\n2023-02 : Lancement d'une nouvelle campagne publicitaire mettant en avant le recyclage des appareils électroniques et électroménagers en magasin.\n2023-03 : Ouverture d'un nouveau magasin phare à Paris, visant à offrir une expérience client améliorée avec des zones de démonstration interactives.\n\nConcurrents:\nBo

In [46]:

events_id = bubble_client.create("Event", [{"Date":ev.date,  "Description": ev.description} for ev in res.recent_events])
events_id = [x["id"] for x in events_id]

In [50]:

competitors_id = []
for competitor in res.competitors:
    data = bubble_client.get("Company", constraints= [BubbleField("Name") == competitor])
    if len(data)>0:
        competitors_id.append(data[0]["_id"])
    else:
        competitors_id.append(bubble_client.create("Company", {"Name": competitor,}))

d = {
    "Name": context["company"],
    "Challenges": res.challenges,
    "Competitors": competitors_id,
    "Description": res.description,
    "Recent events": events_id,
    "website": "www."+context["company"]+".com"
}
d
#bubble_client.update_object("Company", {"Name": competitor,})

{'Name': 'Darty',
 'Challenges': ['Adaptation aux nouvelles technologies et à la digitalisation du commerce.',
  'Concurrence accrue, notamment de la part des géants du e-commerce.',
  "Maintien de la qualité du service après-vente face à l'augmentation du volume de ventes.",
  'Gestion des impacts environnementaux liés à la production et au recyclage des appareils électroniques.'],
 'Competitors': ['1708690119131x131761987332890350',
  '1708690119774x319230027999086500',
  '1708690120400x606675736475846400',
  '1708690121015x200784996865734920'],
 'Description': "Darty est une chaîne de magasins française spécialisée dans la vente de matériel informatique, de téléviseurs, d'électroménager, de matériel audio et vidéo. Fondée en 1957, l'entreprise s'est développée pour devenir l'un des leaders du marché français dans son secteur. Darty est reconnue pour son service après-vente, symbolisé par le 'Contrat de Confiance' qui promet une assistance et des garanties étendues à ses clients.",
 

In [53]:
create_or_update("Company", d)

In [81]:
with open(prompts_path+'prompt_categories_creation_example.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)



class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    sub_categories: Optional[List['InsightCategory']] = Field(default=None, description="Liste des éventuelles sous-catégories.")
    
    def __str__(self):
        if self.sub_categories is not None:
            return self.name + " : \n" + '\n'.join([str(x) for x in self.sub_categories])
        else:
            return self.name
        
InsightCategory.model_rebuild()

class ListInsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Liste des catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, ListInsightCategories)
print(res)

Qualité des produits : Qualité de fabrication : Solidité et durabilitéFinitions et matériauxConformité aux normesFonctionnalité : Performances et efficacitéFacilité d'utilisationConvivialité et designDurabilité : Longévité du produitPièces détachées et réparabilitéGarantie et SAV

Service client : Expérience en magasin : Accueil et conseilsPropreté et organisation du magasinGestion des files d'attenteProcessus de commande en ligne : Facilité de navigation et de rechercheSécurité et confidentialité des donnéesChoix de modes de paiement et de livraisonService de livraison : Délais de livraisonQualité de l'emballage et protection du produitFlexibilité et options de livraisonRetours et remboursements : Facilité et rapidité du processusRemboursement intégral ou échangeService client dédié aux retours et remboursementsSoutien technique : Disponibilité et accessibilitéQualité de l'assistanceSatisfaction client

Prix : Comparaisons avec les concurrents : Prix du produitPromotions et offres spé

In [87]:
def send_categorie(categorie, parents=[], level=1):
    
    bubble_id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
            "Parents": parents,
            "Level": level,
        })

    childrens_id = []
    if categorie.sub_categories is not None:
        for sub_categorie in categorie.sub_categories:
            childrens_id.append(send_categorie(sub_categorie, parents=parents+[bubble_id], level=level+1))

    bubble_client.update_object("Category", bubble_id, {"Childrens": childrens_id})

    return bubble_id
    
for categorie in tqdm(res.categories):
    send_categorie(categorie, parents=[], level=1)

  0%|          | 0/4 [00:00<?, ?it/s]